In [1]:
import pandas as pd
import numpy as np

from RBO import *
from getRecommendations import *
from dataPrep import *

# Read csvs

These are the csv's we cleaned up and combined in step 01. 

In [2]:
ings = pd.read_csv('data/ings.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prods = pd.read_csv('data/prods.csv', encoding = "ISO-8859-1", index_col=0)
#prods = prods.drop_duplicates(subset = 'product')
prod_ing = pd.read_csv('data/prod_ing.csv', encoding = "ISO-8859-1", index_col=0)
prod_ing = prod_ing.drop(['diff'], axis = 1)
#prod_ing = prod_ing.sort_values(['id', 'order'])

print('Number of products: ', len(prods))
print('Number of unique ingredients: ', len(ings))
print('Number of total ingredients: ', len(prod_ing))

Number of products:  15945
Number of unique ingredients:  6899
Number of total ingredients:  446618


In [3]:
df = dataPrep(ings, prods, prod_ing)
#df.to_csv('recommendations/recommender_products.csv')

## Test Product

In [4]:
product = 'Ultra Correction Line Repair Intensive Anti-Wrinkle Concentrate'
indices = pd.Series(prods.index, index=prods['product'])
idx = indices[product]
prods.loc[idx]['product']
#idx
#df.loc[14450]['product']

'Ultra Correction Line Repair Intensive Anti-Wrinkle Concentrate'

# GET RECOMMENDATIONS


In [5]:
df.head(1)

,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,...,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount,id2
0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.0,5.0,16,0,0,0,...,0,0,1,0,0,0,1,0,22,0


In [287]:
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]

#remove water
df['ing#List'] = [remove_values_from_list(i,0) for i in df['ing#List']]

In [7]:
#df.loc[df['id'] == idx]
df['id2'] = df['id']
#df = df.reset_index()
df = df.drop(['id'], axis = 1)
df = df.rename(columns={'index': 'id'})
df.head(3)

,ingList,ing#List,product,brand,price,rating,ratingsCount,cat_Acne&BlemishTreatments,cat_Ampoules,cat_BodySkincare,...,note_Brightening,note_ContainsAlcohol,note_ContainsParaben,note_ContainsSulfate,note_ParabenFree,note_PromotesWoundHealing,note_SulfateFree,note_UVProtection,ingCount,id2
0,"[Water, Hydrogenated Polyisobutene, Butylene G...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Naturally Gentle Eye Makeup Remover,Clinique,20.00,5.0,16,0,0,0,...,0,0,1,0,0,0,1,0,22,0
1,"[Water, Glycerin, Cetearyl Alcohol, Petrolatum...","[0, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32...",Foot Cream Norwegian Formula,Neutrogena,23.82,4.9,84,0,0,0,...,1,0,1,0,0,1,1,0,21,2
2,"[Water, Glycereth-26, Alcohol, Butylene Glycol...","[0, 39, 40, 2, 41, 42, 43, 44, 45, 46, 47, 48,...",Essential Power Skin Refiner_Moisture,Laneige,28.20,5.0,12,0,0,0,...,1,1,0,0,1,1,1,0,38,3


In [18]:
#id: 1961
product ='Facial Treatment Essence'
ingTest = df[df['product'].str.match(product)]
ingTest = ingTest['ingList']
print(*ingTest, sep='\n')

['Saccharomyces Ferment Filtrate', 'Butylene Glycol', 'Pentylene Glycol', 'Water', 'Sodium Benzoate', 'Methylparaben', 'Sorbic Acid']


## Cosine similarity

Reference [here](https://www.datacamp.com/community/tutorials/recommender-systems-python)

In [19]:
%%time

recommended = get_COS_recommendations(df,product)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/COS.csv')

Wall time: 33.8 s


In [20]:
topTen

,product,ingList,commonIng
14839,Facial Treatment Repair C,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
14515,Facial Treatment Mask,"[Water, Saccharomyces Ferment Filtrate, Butyle...",4
3873,Advanced Eye Treatment Film,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
3473,Ultimate Revival Serum,"[Water, Saccharomyces Ferment Filtrate, Butyle...",6
14951,LXP Ultimate Serum,"[Water, Saccharomyces Ferment Filtrate, Butyle...",6
14629,Facial Treatment Clear Lotion,"[Water, Saccharomyces Ferment Filtrate, Butyle...",5
15103,Pitera Mid-Day Essence,"[Galactomyces Ferment Filtrate, Butylene Glyco...",6
15180,Whitening Source Clear Lotion,"[Water, Dipropylene Glycol, Saccharomyces Ferm...",5
14876,Skin Signature Mask 3-D Redefining Mask,"[Water, Saccharomyces Ferment Filtrate, Glycer...",5
4463,Fresh ?n Fair Skin Refresher,"[Water, Alcohol Denat., Propylene Glycol, Glyc...",3


## Average Overlap

In [34]:
#df['ing#List'] = [ast.literal_eval(i) for i in df['ing#List']]
indices = pd.Series(df.index, index=df['product']).drop_duplicates()
idx = indices[product]

itemLookup = df.loc[idx]['ing#List']
items = df['ing#List']
sim = [average_overlap(itemLookup,i) for i in items] ##change method here
sim_scores = list(enumerate(sim))

# Sort the products based on the similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

# Get the scores of the 10 most similar products
sim_scores = sim_scores[1:11]

# Get the product indices
product_indices = [i[0] for i in sim_scores]

recommended = df[['product', 'ingList']].iloc[product_indices]

topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/AO.csv')

In [35]:
topTen

,product,ingList,commonIng
14839,Facial Treatment Repair C,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
3873,Advanced Eye Treatment Film,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
3473,Ultimate Revival Serum,"[Water, Saccharomyces Ferment Filtrate, Butyle...",6
14951,LXP Ultimate Serum,"[Water, Saccharomyces Ferment Filtrate, Butyle...",6
14515,Facial Treatment Mask,"[Water, Saccharomyces Ferment Filtrate, Butyle...",4
7649,Sake Hydrating and Brightening Essence,"[Saccharomyces Ferment Filtrate, Methyl Glucet...",4
1040,Cleanance Cleansing Water,"[Water, Butylene Glycol, Pentylene Glycol, PEG...",3
2611,Instant Eye Makeup Remover,"[Water, Butylene Glycol, Pentylene Glycol, PEG...",3
2640,Pore Milk,"[Water, Butylene Glycol, Pentylene Glycol, Gly...",3
2711,Pore Lotion,"[Water, Butylene Glycol, Pentylene Glycol, Gly...",3


## RBO similarity measure

Based on code found [here](https://github.com/dlukes/rbo/blob/master/rbo.py)

In [ ]:
df

In [23]:
#%%time

recommended = get_RBO_recommendations(df, product, rbo_min)
topTen = pd.DataFrame(recommended)
commonItems(product, topTen, df)
recommended.to_csv('recommendations/RBO.csv')

In [24]:
topTen

,product,ingList,commonIng
2296,Time Revolution The First Treatment Essence In...,"[Saccharomyces Ferment Filtrate, Bifida Fermen...",2
14839,Facial Treatment Repair C,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
3873,Advanced Eye Treatment Film,"[Saccharomyces Ferment Filtrate, Water, Glycer...",6
911,Time Revolution First Treatment Essence,"[Saccharomyces Ferment Filtrate, Bifida Fermen...",4
2494,Time Revolution The First Treatment Essence (I...,"[Saccharomyces Ferment Filtrate, Bifida Fermen...",4
7649,Sake Hydrating and Brightening Essence,"[Saccharomyces Ferment Filtrate, Methyl Glucet...",4
14660,First Treatment Essence Mask,"[Saccharomyces Ferment Filtrate, Water, Propan...",3
7846,Original Tube,"[Petrolatum, Lanolin, Cetyl Esters, Theobroma ...",0
10949,Saemmul Kiss Lip Balm,"[Petrolatum, Ceresin, Hydrogenated Polyisobute...",2
2465,Bio Essence Intensive Conditioning,"[Saccharomyces Ferment Filtrate, Niacinamide, ...",2
